# Example creating custom models with GPT-2




In [1]:
# Import various things
import neural_nlp
from neural_nlp import score
from neural_nlp.models import model_pool, model_layers, implementations
from brainscore.utils import LazyLoad
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config

/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
def gpt2_test():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained(
        "gpt2", output_hidden_states=True
    )
    wrapper = implementations._PytorchTransformerWrapper(
        identifier="gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12))),
        sentence_average=implementations.word_last
    )
    return wrapper

In [3]:
model_pool["gpt2-test"] = LazyLoad(lambda: gpt2_test())
model_layers["gpt2-test"] = list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12)))
assert model_layers["gpt2-test"] == model_layers["gpt2"]


In [4]:
print(GPT2Tokenizer.from_pretrained("gpt2"))

In [5]:
# Run and evaluate the model generated in gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="gpt2-test")

<xarray.Score (layer: 13, aggregation: 2)>
array([[0.04500979, 0.10728886],
       [0.60563093, 0.13442728],
       [0.67513002, 0.02300425],
       [0.63005166, 0.07155539],
       [0.5024158 , 0.08306366],
       [0.48248263, 0.12891596],
       [0.50950414, 0.09476689],
       [0.559599  , 0.10482938],
       [0.63031937, 0.18250217],
       [0.72617442, 0.14373017],
       [0.80979421, 0.12343126],
       [0.88677038, 0.05182417],
       [0.9121889 , 0.11140106]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.11'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.01433...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        gpt2-test
    benchmark:    Pereira2018-encoding

In [6]:
# default_model = GPT2Mode.from_pretrained("gpt2", output_hidden_states=True)
# default_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# default_identifier = "gpt2"
# def model_test(identifier="gpt2-test", model=default_model, tokenizer=default_tokenizer):
#     implementations

def untrained_gpt2_test():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    config = GPT2Config.from_pretrained("gpt2")
    
    # Schrimpf et al. initialize the model this way
    # See lines approximately 1325-1330 in implementations.py
    model = GPT2Model(config)
    state_dict = model.state_dict()  # Capture the initial random weights
    
    # Force the initial random weights back onto the pre-trained model
    model = GPT2Model.from_pretrained("gpt2", output_hidden_states=True, state_dict=state_dict)
    model = model.float()
    
    wrapper = implementations._PytorchTransformerWrapper(
        identifier="untrained-gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12))),
        sentence_average=implementations.word_last
    )
    return wrapper

In [7]:
model_pool["untrained-gpt2-test"] = LazyLoad(lambda: untrained_gpt2_test())
model_layers["untrained-gpt2-test"] = list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12)))

In [8]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-gpt2-test")

<xarray.Score (layer: 13, aggregation: 2)>
array([[0.04223047, 0.03959577],
       [0.81282733, 0.06704036],
       [0.86791751, 0.08255504],
       [0.87549202, 0.10991611],
       [0.86945091, 0.08111384],
       [0.88217028, 0.10755938],
       [0.88257703, 0.10618529],
       [0.88984818, 0.13134028],
       [0.88688   , 0.12774002],
       [0.88666312, 0.10500984],
       [0.87644635, 0.08464181],
       [0.8730716 , 0.07575206],
       [0.85522824, 0.06460767]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.11'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.01345...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-gpt2-test
    benchmark:    Pereira2018-encoding

In [30]:
# Use different embeddings (dimensions) for gpt-2 layers. Consistently use 24 layers
n_layer = 24
def untrained_n_embedding_gpt2_test(embeddings):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    config = GPT2Config.from_pretrained("gpt2", output_hidden_states=True)
    config.n_layer = n_layer
    config.n_embed = embeddings
    model = GPT2Model(config)

    wrapper = implementations._PytorchTransformerWrapper(
        identifier=f"untrained-{embeddings}-embedding-gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(n_layer))),
        sentence_average=implementations.word_last
    )
    return wrapper


In [31]:
# Set up several untrained models of different number of embeddings
for embeddings in [10, 20, 50, 100, 500, 768, 1000]:
    
    model_pool[f"untrained-{embeddings}-embedding-gpt2-test"] = (
        lambda e: LazyLoad(
            lambda: untrained_n_embedding_gpt2_test(e)
        )
    )(embeddings)
    model_layers[f"untrained-{embeddings}-embedding-gpt2-test"] = list(
        ("drop",) + tuple(f"encoder.h.{i}" for i in range(n_layer))
    )

In [32]:
print(model_pool.keys())

dict_keys(['sentence-length', 'word-position', 'random-embedding', 'skip-thoughts', 'skip-thoughts-untrained', 'lm_1b', 'lm_1b-untrained', 'word2vec', 'word2vec-untrained', 'glove', 'glove-untrained', 'transformer', 'transformer-untrained', 'ETM', 'ETM-untrained', 'bert-base-uncased', 'bert-base-multilingual-cased', 'bert-large-uncased', 'bert-large-uncased-whole-word-masking', 'openaigpt', 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'transfo-xl-wt103', 'xlnet-base-cased', 'xlnet-large-cased', 'xlm-mlm-en-2048', 'xlm-mlm-enfr-1024', 'xlm-mlm-xnli15-1024', 'xlm-clm-enfr-1024', 'xlm-mlm-100-1280', 'roberta-base', 'roberta-large', 'distilroberta-base', 'distilbert-base-uncased', 'ctrl', 'albert-base-v1', 'albert-base-v2', 'albert-large-v1', 'albert-large-v2', 'albert-xlarge-v1', 'albert-xlarge-v2', 'albert-xxlarge-v1', 'albert-xxlarge-v2', 't5-small', 't5-base', 't5-large', 't5-3b', 't5-11b', 'xlm-roberta-base', 'xlm-roberta-large', 'bert-base-uncased-untrained', 'bert-b

In [33]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-10-embedding-gpt2-test")

token features:  79%|█████████████████████▍     | 54/68 [00:24<00:06,  2.23it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 68/68 [00:00<00:00, 85.08it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.69it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 84.91it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 265.21it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 40/40 [00:00<00:00, 91.03it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 270.92it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 68/68 [00:00<00:00, 85.89it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 271.63it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 85.14it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.68it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 87.77it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.75it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 49/49 [00:00<00:00, 82.62it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.61it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 63/63 [00:00<00:00, 83.81it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 275.34it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 85.56it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 273.19it/s]
Using cls_token, but 

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 271.82it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 90.79it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.59it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 87.56it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.62it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 55/55 [00:00<00:00, 105.70it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 291.71it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00<00:00, 97.31it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 290.07it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00

token features: 100%|██████████████████████████| 67/67 [00:00<00:00, 104.36it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 270.55it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 41/41 [00:00<00:00, 78.51it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 279.52it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 50/50 [00:00<00:00, 92.55it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 268.24it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 55/55 [00:00<00:00, 85.56it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 275.20it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 55/55 [00:00<00:00, 85.38it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.04it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 84.67it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 287.29it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00<00:00, 86.07it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.29it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00<00:00, 91.66it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.63it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 93.16it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 291.08it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 85.51it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.70it/s]
Using cls_token, but 

...

cross-validation:  80%|█████████████████████▌     | 4/5 [01:34<00:23, 23.58s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:57<00:00, 23.58s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:44<07:20, 88.06s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:44<07:20, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:06<00:25,  6.37s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:12<00:19,  6.37s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:19<00:12,  6.35s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:25<00:06,  6.35s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:31<00:00,  6.34s/it]

{'experiment': '243sentences', 'atlas': 'language'}:  60%|▌| 6/10 [07:16<04:35, 
{'experiment': '243sentences', 'atlas': 'DMN'}:  60%|▌| 6/10 [07:16<04:35, 68.98

cross-validation: 



cross-validation: 100%|███████████████████████████| 5/5 [02:51<00:00, 34.32s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:22<11:58, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:22<11:58, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|█████▍                     | 1/5 [00:04<00:17,  4.25s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|██████████▊                | 2/5 [00:08<00:12,  4.



cross-validation: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.08s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:50<07:35, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:50<07:35, 50.65

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:07<00:31,  7.86s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:15<00:23,  7.90s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:23<00:15,  7.92s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:31<00:07,  7.94s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:39<00:00,  7.93s/it]

{'experiment': '384sentences', 'atlas': 'DMN'}:  20%|▏| 2/10 [01:30<05:54, 44.33
{'experiment': '384sentences', 'atlas': 'visual'}:  20%|▏| 2/10 [01:30<05:54, 44

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]/home/rwoo/Documen

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.76s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:27<00:51, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:27<00:51, 51.37s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:14<00:56, 14.20s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:28<00:42, 14.17s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:42<00:28, 14.20s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:56<00:14, 14.20s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:11<00:00, 14.26s/it]

{'experiment': '243sentences', 'atlas': 'MD'}: 100%|█| 10/10 [10:39<00:00, 63.95
layers: 100%|████████████████████████████████| 25/25 [4:34:24<00:00, 658.60s/it]


<xarray.Score (layer: 25, aggregation: 2)>
array([[0.03872582, 0.11402085],
       [0.78971892, 0.09692257],
       [0.8443061 , 0.05445144],
       [0.87057063, 0.11833863],
       [0.88990241, 0.05671713],
       [0.90485135, 0.10153557],
       [0.89898959, 0.05296617],
       [0.88986379, 0.07374882],
       [0.88922881, 0.10531158],
       [0.91897324, 0.14983308],
       [0.90957477, 0.15278893],
       [0.88874723, 0.13065168],
       [0.89102288, 0.13137768],
       [0.90803605, 0.11847565],
       [0.90466126, 0.12600947],
       [0.89139833, 0.11648843],
       [0.89153821, 0.11722361],
       [0.89817201, 0.13777728],
       [0.89971122, 0.13742477],
       [0.9069629 , 0.13471655],
       [0.90708145, 0.1287673 ],
       [0.93579999, 0.13363783],
       [0.9296681 , 0.11621894],
       [0.92700357, 0.1387064 ],
       [0.93294663, 0.13229929]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.01233...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-10-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [34]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-20-embedding-gpt2-test")

layers:   0%|                                            | 0/25 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 66/66 [00:00<00:00, 88.33it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.89it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 72/72 [00:00<00:00, 104.53it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.01it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 98.95it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.80it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 86.86it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 276.93it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 88.62it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.88it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 67/67 [00:00<00:00, 90.12it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 291.25it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 104.72it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.43it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 61/61 [00:00<00:00, 85.84it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.01it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00<00:00, 83.58it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 290.10it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 82.58it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.41it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00<00:00, 87.07it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 288.82it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 92.37it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.55it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 90.44it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.33it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00<00:00, 86.96it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.85it/s]
Using cls_token, but 

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.94it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 78.55it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 277.53it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 78.87it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.39it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<00:00, 88.17it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.87it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 88.72it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.58it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 85.25it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.94it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 90.99it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.20it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 83.59it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.70it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 88.13it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.01it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 37/37 [00:00<00:00, 88.80it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.35it/s]
Using cls_token, but 

...



cross-validation:  60%|████████████████▏          | 3/5 [01:43<01:08, 34.34s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [02:17<00:34, 34.29s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [02:51<00:00, 34.36s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:24<12:02, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:24<12:02, 

cross-validation:   0%|                                   | 0/5 [00:00<?,



cross-validation:  60%|████████████████▏          | 3/5 [00:30<00:20, 10.04s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [00:40<00:10, 10.11s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.13s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:50<07:38, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:50<07:38, 50.92

cross-validation:   0%|                                   | 0/5 [00:00<?,

cross-validation: 100%|███████████████████████████| 5/5 [01:36<00:00, 19.32s/it]

{'experiment': '243sentences', 'atlas': 'visual'}:  80%|▊| 8/10 [09:14<02:15, 67
{'experiment': '243sentences', 'atlas': 'auditory'}:  80%|▊| 8/10 [09:14<02:15, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:02<00:10,  2.70s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:05<00:08,  2.70s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:08<00:05,  2.71s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:10<00:02,  2.70s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.72s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:28<00:50, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:28<00:50, 50.92s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation: 



cross-validation: 100%|███████████████████████████| 5/5 [00:20<00:00,  4.19s/it]

{'experiment': '384sentences', 'atlas': 'auditory'}:  40%|▍| 4/10 [04:43<07:01, 
{'experiment': '384sentences', 'atlas': 'MD'}:  40%|▍| 4/10 [04:43<07:01, 70.31s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:23<01:34, 23.54s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:47<01:10, 23.52s/it]

cross-validation:  60%|████████████████▏          | 3/5 [01:10<00:47, 23.61s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [01:34<00:23, 23.59s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:58<00:00, 23.64s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:42<07:18, 87.70s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:42<07:18, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.01443505, 0.07044489],
       [0.74201145, 0.06103926],
       [0.82170769, 0.20037037],
       [0.84818345, 0.0803312 ],
       [0.86078558, 0.078837  ],
       [0.89149817, 0.13604651],
       [0.87487307, 0.10822428],
       [0.90259087, 0.10967738],
       [0.87831357, 0.15388197],
       [0.9023393 , 0.14356523],
       [0.91863991, 0.12113445],
       [0.91530603, 0.11822879],
       [0.91036752, 0.11607576],
       [0.9066447 , 0.13300864],
       [0.90929944, 0.12292178],
       [0.91481066, 0.14881989],
       [0.92369131, 0.13603618],
       [0.91032872, 0.14612877],
       [0.89732672, 0.13395828],
       [0.8956142 , 0.12990409],
       [0.89085038, 0.12291545],
       [0.8944899 , 0.12157745],
       [0.90079962, 0.13063956],
       [0.89262368, 0.13887001],
       [0.89012281, 0.13025573]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00459...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-20-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [35]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-50-embedding-gpt2-test")

layers:   0%|                                            | 0/25 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 66/66 [00:00<00:00, 91.46it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.94it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 72/72 [00:00<00:00, 105.39it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.31it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 104.43it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.17it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 90.86it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.95it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 90.73it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 288.97it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 67/67 [00:00<00:00, 88.93it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.44it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 102.03it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.32it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 61/61 [00:00<00:00, 84.79it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.74it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00<00:00, 85.92it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.21it/s]
Using cls_token, but 

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.39it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 84.64it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.47it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00<00:00, 83.68it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.30it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 93.62it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.08it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 89.17it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.74it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00

token features: 100%|███████████████████████████| 44/44 [00:00<00:00, 84.70it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 292.68it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 82.41it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 291.99it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 82.69it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.76it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<00:00, 91.20it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.11it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 92.15it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.37it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 84.63it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.35it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 89.66it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 290.89it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 85.97it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 292.52it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 88.41it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.69it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 37/37 [00:00<00:00, 85.35it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 292.34it/s]
Using cls_token, but 

...



cross-validation:  40%|██████████▊                | 2/5 [01:07<01:41, 33.67s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  60%|████████████████▏          | 3/5 [01:41<01:07, 33.75s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [02:15<00:33, 33.85s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRCons



cross-validation:  40%|██████████▊                | 2/5 [00:20<00:31, 10.35s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  60%|████████████████▏          | 3/5 [00:31<00:20, 10.36s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [00:41<00:10, 10.28s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRCons

cross-validation:  20%|█████▍                     | 1/5 [00:19<01:17, 19.29s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:38<00:57, 19.26s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:57<00:38, 19.33s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [01:17<00:19, 19.33s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:36<00:00, 19.34s/it]

{'experiment': '243sentences', 'atlas': 'visual'}:  80%|▊| 8/10 [09:12<02:15, 67
{'experiment': '243sentences', 'atlas': 'auditory'}:  80%|▊| 8/10 [09:12<02:15, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:02<00:10,  2.69s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:05<00:08,  2.70s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:08<00:05,  2.70s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:10<00:02,  2.70s/it]

cross-validation:



cross-validation:  80%|█████████████████████▌     | 4/5 [00:16<00:04,  4.21s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [00:21<00:00,  4.21s/it]

{'experiment': '384sentences', 'atlas': 'auditory'}:  40%|▍| 4/10 [04:46<07:05, 
{'experiment': '384sentences', 'atlas': 'MD'}:  40%|▍| 4/10 [04:46<07:05, 70.99s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:23<01:33, 23.38s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:46<01:09, 23.15s/it]

cross-validation:  60%|████████████████▏          | 3/5 [01:09<00:46, 23.17s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [01:32<00:23, 23.02s/it]

cro

<xarray.Score (layer: 25, aggregation: 2)>
array([[0.00283364, 0.03440709],
       [0.77787181, 0.0696091 ],
       [0.84637071, 0.05412396],
       [0.86249201, 0.10005908],
       [0.85287524, 0.06516104],
       [0.88167335, 0.09037197],
       [0.8868236 , 0.13516442],
       [0.84660862, 0.13251538],
       [0.86125045, 0.10251875],
       [0.87037009, 0.1149067 ],
       [0.87375385, 0.13195403],
       [0.88536236, 0.1393623 ],
       [0.91314191, 0.13739212],
       [0.91910564, 0.14139671],
       [0.92833929, 0.13915581],
       [0.93618848, 0.10608457],
       [0.93124467, 0.09316814],
       [0.94104967, 0.10320042],
       [0.93200857, 0.1305396 ],
       [0.91942813, 0.1040272 ],
       [0.92482263, 0.06777612],
       [0.90941531, 0.0987366 ],
       [0.90757341, 0.09042905],
       [0.90556225, 0.08243331],
       [0.92334224, 0.07345216]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[0.00090...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-50-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [36]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-100-embedding-gpt2-test")

layers:   0%|                                            | 0/25 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 66/66 [00:00<00:00, 89.03it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.90it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 72/72 [00:00<00:00, 103.86it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 276.81it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 102.87it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.75it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 62/62 [00:00<00:00, 86.91it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 278.21it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<

token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 91.10it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.66it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 67/67 [00:00<00:00, 90.25it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.88it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████████████████████| 56/56 [00:00<00:00, 101.75it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.32it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 61/61 [00:00<00:00, 84.74it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.41it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 48/48 [00:00<00:00, 82.41it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 279.92it/s]
Using cls_token, but 

token features: 100%|███████████████████████████| 46/46 [00:00<00:00, 86.47it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 281.85it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 42/42 [00:00<00:00, 85.02it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 286.50it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 93.28it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 287.94it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00<00:00, 92.82it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 290.85it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 53/53 [00:00<00:00, 88.57it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.26it/s]
Using cls_token, but 


token features: 100%|███████████████████████████| 45/45 [00:00<00:00, 79.80it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.65it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 80.12it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 283.90it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 60/60 [00:00<00:00, 89.49it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 280.34it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 90.59it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.17it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 54/54 [00:00<00:00, 91.59it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 285.21it/s]
Using cls_token, but

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 282.19it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 56/56 [00:00<00:00, 89.77it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.83it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 57/57 [00:00<00:00, 82.40it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 284.06it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 52/52 [00:00<00:00, 86.25it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 274.95it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 37/37 [00:00<00:00, 88.43it/s]

layer packaging: 100%|█████████████████████████| 25/25 [00:00<00:00, 289.02it/s]
Using cls_token, but it is not set yet.

token features: 100%|███████████████████████████| 47/47 [00:00

...



cross-validation:  60%|████████████████▏          | 3/5 [01:43<01:08, 34.49s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [02:18<00:34, 34.57s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [02:52<00:00, 34.50s/it]

{'experiment': '384sentences', 'atlas': 'visual'}:  30%|▎| 3/10 [04:24<12:03, 10
{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|▎| 3/10 [04:24<12:03, 

cross-validation:   0%|                                   | 0/5 [00:00<?,



cross-validation:  60%|████████████████▏          | 3/5 [00:30<00:20, 10.15s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  80%|█████████████████████▌     | 4/5 [00:40<00:10, 10.16s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.16s/it]

{'experiment': '384sentences', 'atlas': 'language'}:  10%| | 1/10 [00:51<07:39, 
{'experiment': '384sentences', 'atlas': 'DMN'}:  10%| | 1/10 [00:51<07:39, 51.05

cross-validation:   0%|                                   | 0/5 [00:00<?,

cross-validation: 100%|███████████████████████████| 5/5 [01:36<00:00, 19.36s/it]

{'experiment': '243sentences', 'atlas': 'visual'}:  80%|▊| 8/10 [09:14<02:15, 67
{'experiment': '243sentences', 'atlas': 'auditory'}:  80%|▊| 8/10 [09:14<02:15, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:02<00:10,  2.70s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:05<00:08,  2.75s/it]

cross-validation:  60%|████████████████▏          | 3/5 [00:08<00:05,  2.75s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [00:10<00:02,  2.74s/it]

cross-validation: 100%|███████████████████████████| 5/5 [00:13<00:00,  2.74s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|▉| 9/10 [09:28<00:51, 
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|▉| 9/10 [09:28<00:51, 51.08s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation: 



cross-validation: 100%|███████████████████████████| 5/5 [00:20<00:00,  4.18s/it]

{'experiment': '384sentences', 'atlas': 'auditory'}:  40%|▍| 4/10 [04:44<07:03, 
{'experiment': '384sentences', 'atlas': 'MD'}:  40%|▍| 4/10 [04:44<07:03, 70.62s

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|█████▍                     | 1/5 [00:23<01:33, 23.29s/it]

cross-validation:  40%|██████████▊                | 2/5 [00:46<01:09, 23.23s/it]

cross-validation:  60%|████████████████▏          | 3/5 [01:09<00:46, 23.27s/it]

cross-validation:  80%|█████████████████████▌     | 4/5 [01:33<00:23, 23.34s/it]

cross-validation: 100%|███████████████████████████| 5/5 [01:56<00:00, 23.31s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|▌| 5/10 [06:41<07:16, 87.30s
{'experiment': '243sentences', 'atlas': 'language'}:  50%|▌| 5/10 [06:41<07:16, 

cross-validation:   0%|                                   | 0/5 [00:00<?, ?it/s]

cross-validation

<xarray.Score (layer: 25, aggregation: 2)>
array([[-0.00237707,  0.06207606],
       [ 0.80784967,  0.07298576],
       [ 0.8568618 ,  0.13641384],
       [ 0.86069286,  0.16049764],
       [ 0.88624691,  0.14548278],
       [ 0.89373423,  0.13948103],
       [ 0.89521737,  0.14038527],
       [ 0.88493891,  0.15713547],
       [ 0.88044761,  0.15831589],
       [ 0.88935238,  0.19472925],
       [ 0.89724713,  0.16704635],
       [ 0.90224772,  0.14370174],
       [ 0.91100208,  0.13880974],
       [ 0.91271786,  0.14585289],
       [ 0.90579359,  0.13951717],
       [ 0.90321663,  0.11289885],
       [ 0.90974557,  0.08625501],
       [ 0.91637223,  0.08904756],
       [ 0.92875084,  0.11165338],
       [ 0.93812923,  0.10904582],
       [ 0.92809192,  0.10555831],
       [ 0.92753711,  0.11030191],
       [ 0.92975045,  0.11877295],
       [ 0.92073381,  0.10979289],
       [ 0.9275986 ,  0.08192335]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.23'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 25, aggregation: 2)>\narray([[-0.0007...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        untrained-100-embedding-gpt2-test
    benchmark:    Pereira2018-encoding

In [ ]:
# Run and evaluate the mode generated in untrained_gpt2_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME THE FIRST TIME YOU RUN IT (~ 2 hours)

score(benchmark="Pereira2018-encoding", model="untrained-500-embedding-gpt2-test")